# Módulo 8: Orquestación Secuencial de Agentes

En este ejercicio, utilizarás el patrón de orquestación secuencial para crear un flujo de trabajo con tres agentes especializados que colaboran para analizar un problema técnico y proponer un plan de acción.

### Cargar librerías y variables de entorno

In [ ]:
import asyncio
from dotenv import load_dotenv, find_dotenv
from typing import cast
from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential

load_dotenv(find_dotenv(usecwd=True))

### Definir las Instrucciones de los Agentes

Cada agente tiene un rol específico en el flujo de trabajo: extractor de requisitos, evaluador de riesgos y planificador de acciones.

In [ ]:
extractor_instructions="""
You are an Aerospace Systems Engineer. Extract technical requirements, subsystems, and constraints.
Return clear and objective technical summary.
"""

risk_instructions="""
You are an Aviation Safety Engineer. Evaluate technical risks, prioritize criticality, and flag missing data.
"""

planner_instructions="""
You are an Aerospace Technical Planner. Propose concrete next action and suggested owner.
"""

### Crear y ejecutar la orquestación secuencial

In [ ]:
async def process_technical_case(prompt):
    print("Starting sequential agent orchestration...")
    credential = AzureCliCredential()
    async with AzureAIAgentClient(async_credential=credential) as chat_client:
        print("Creating requirements extractor agent...")
        extractor = chat_client.create_agent(
            instructions=extractor_instructions,
            name="requirements_extractor",
        )
        print("Requirements extractor created")
        
        print("Creating risk assessor agent...")
        risk = chat_client.create_agent(
            instructions=risk_instructions,
            name="risk_assessor",
        )
        print("Risk assessor created")
        
        print("Creating action planner agent...")
        planner = chat_client.create_agent(
            instructions=planner_instructions,
            name="action_planner",
        )
        print("Action planner created")

        print("Building sequential workflow...")
        workflow = SequentialBuilder().participants([extractor, risk, planner]).build()
        print("Workflow built")

        outputs: list[list[ChatMessage]] = []
        print(f"Processing technical case: {prompt}")
        async for event in workflow.run_stream(f"Technical input: {prompt}"):
            if isinstance(event, WorkflowOutputEvent):
                outputs.append(cast(list[ChatMessage], event.data))

        if outputs:
            print("\nOrchestration results:")
            for i, msg in enumerate(outputs[-1], start=1):
                name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
                print(f"{'=' * 20} {name} {'=' * 20}\n{msg.text}\n")
        print("Orchestration completed")

# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Premature wear on main shaft bearings in A320 fleet after 2,000 cycles; suspected FOD contamination."
await process_technical_case(user_prompt)
print("Technical case processing finished")

In [ ]:
# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Incremento de vibraciones en fase de ascenso, canal de aviónica reporta fallos intermitentes ARINC 429 en ADAHRS; sin registros de fallo duro. ¿Plan?"
await process_technical_case(user_prompt)
print("Technical case processing finished")

In [ ]:
# Enter technical case here
print("Starting technical case processing...")
user_prompt = "Solicito trazabilidad digital completa (CoC, FAIR, lotes, heat treatment) integrada con PLM/MES para carcasas de compresor del programa UltraFan."
await process_technical_case(user_prompt)
print("Technical case processing finished")